<a href="https://colab.research.google.com/github/Roger-G/Data-augmentation/blob/master/Build_fine_tune_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import package

In [0]:
!pip install transformers
!pip install pysnooper
# !pip install torch
"Fine-tuning BertMasked Model with labeled dataset"
from __future__ import absolute_import, division, print_function
import argparse
import logging
import os
import random
import csv
import pandas as pd
from IPython.display import clear_output
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, TensorDataset, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import trange
import shutil
from pathlib import Path
from transformers import BertForMaskedLM
from transformers import BertTokenizer
from transformers import AdamW

import nltk
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt') 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
absFilePath = 'drive/My Drive/Data/Bert_ data augmentation-master/'
logger = logging.getLogger(__name__)
train_number = 0
clear_output()

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocess

In [0]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, init_ids, input_ids, input_mask, segment_ids, masked_lm_labels, token_idx):
        self.init_ids = init_ids
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.masked_lm_labels = masked_lm_labels
        self.token_idx = token_idx


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

## Conver_example_to_features

In [0]:

def index_value(list_long, list_index, list_value):
    if len(list_value) == 1:
      for i in list_index:
        list_long[i] = list_value[0]
    else:
      for i in list_index:
        list_long[i] = list_value[i]
    return list_long

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}
    print('label_map', label_map)
    features = []
    dupe_factor = 5
    masked_lm_prob = 0.15
    rng = random.Random(123)
    max_predictions_per_seq = 20
    a = examples
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a) # (aaaaa, 1), aaaa is text_a
        tokens_b = None
        if len(tokens_a) > max_seq_length - 2:  # maxlength = [cls]+token_length + [sep]
            tokens_a = tokens_a[:(max_seq_length - 2)]
        # print('label', example.label, len(example.label))
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"] # tokens_a is a token of sentence
        # print('tokens', tokens)
        #################################################################################
        pos = nltk.pos_tag(tokens) 
        # print('pos', pos)
        index_vb = [ind for ind, (word, tag) in enumerate(pos) if tag in ('VBD', 'VB', 'VBG', 'VBN', 'JJ', 'JJR', 'JJS') 
                    and (word not in stopwords.words('english')) and (word not in ('[CLS]', '[PAD]', '[SEP]'))]
      
        ######################################################################################            
        s = example.label
        try:
            label_id = label_map[s]
        except KeyError:

            label_id = label_map[s.strip()]

        segment_ids = [label_id] * len(tokens) # use this label to make label_id and segment_ids # here maybe is the contextual expression
        masked_lm_labels = [-100]*max_seq_length
        
        # print('segment_ids', segment_ids)
        cand_indexes = []
        for (i, token) in enumerate(tokens):
            if token not in ["[CLS]", '[PAD]', "[SEP]"]:
              if token not in stopwords.words('english'):
                cand_indexes.append(i)
        rng.shuffle(cand_indexes) # 随机替换词为mask，为下面做准备
        # print('cand_indexes', cand_indexes)
        len_cand = len(cand_indexes)
        output_tokens = list(tokens) ## contain '[CLS]' and '[SEP]'
        num_to_predict = min(max_predictions_per_seq, ##  max_predictions_per_seq=20 最多预测20个
                             max(1, int(round(len(tokens) * masked_lm_prob)))) # 最多替换tokens的50%
        # print('num_to_predict', num_to_predict)
        if len(index_vb) > num_to_predict:

            index_vb_new = random.sample(index_vb, num_to_predict)
            # print('masked_lms_pos_test', index_vb_new)
            masked_lms_pos = index_vb_new
            output_tokens = index_value(output_tokens, index_vb_new, ['[MASK]'])
            all_tokens_ids = tokenizer.convert_tokens_to_ids(tokens) #被mask的位置的原来的token的ids
            masked_lm_labels = index_value(masked_lm_labels, index_vb_new, all_tokens_ids)
            
            # token_idx[index] = tokenizer.convert_tokens_to_ids([tokens[index]])[0] #被mask的位置的原来的token的ids
            # masked_origin_tokens = index_value(masked_lm_labels, index_vb_new, 1)
            # print('output_tokens', output_tokens)
            
        else:
            output_tokens = index_value(output_tokens, index_vb, ['[MASK]'])
            masked_lms_pos = index_vb

            covered_indexes = set(index_vb)
            all_tokens_ids = tokenizer.convert_tokens_to_ids(tokens) 
            # print('all_tokens_ids', all_tokens_ids)
            # all_tokens_ids = all_tokens_ids[0]
            masked_lm_labels = index_value(masked_lm_labels, index_vb, all_tokens_ids)
            for index in [indx for indx in cand_indexes if indx not in index_vb]: # cand_indexes 是除 [CLS] 和 ['SEP'] 的token 的index集合, 但是已经被shuffle了, sentence
                if len(masked_lms_pos) >= num_to_predict:
                    break
                if index in covered_indexes:
                    continue
                covered_indexes.add(index)
                masked_token = None # 处理完一个词， should be reset for every token
                # 80% of the time, replace with [MASK]，这个句子的token的80%换成[MASK], 10% keep original, 10% random 
                if rng.random() < 0.8:
                    masked_token = "[MASK]"
                    # output_tokens[index] = masked_token # mask 位置的token
                    masked_lms_pos.append(index)
                    # print('masked_lms_pos', masked_lms_pos)
                    masked_lm_labels[index] = tokenizer.convert_tokens_to_ids([tokens[index]])[0]  #被mask的位置的原来的token的ids
                else:
                    # 10% of the time, keep original
                    # print('rng.random()', rng.random())
                    if rng.random() < 0.5:
                        masked_token = tokens[index]
                    # 10% of the time, replace with random word
                    else:
                        masked_token = tokens[cand_indexes[rng.randint(0, len_cand - 1)]]
                    # print('masked_token', masked_token)
                output_tokens[index] = masked_token

                
            # print('masked_lm_labels and ', masked_lm_labels)
            # output_tokens[index] = masked_token # mask 位置的token
            # masked_lms_pos.append(index) ## 被mask的token 在这句话的位置
        
        # print('masked_lm_labels', masked_lm_labels)
        # print('original tokens before training', tokens)
        init_ids = tokenizer.convert_tokens_to_ids(tokens) # original tokens
        token_idx = masked_lms_pos # token de position
        # print('output_tokens', output_tokens, type(output_tokens))

        input_ids = tokenizer.convert_tokens_to_ids(output_tokens) # ids : original tokens with [MASK] 
        input_mask = [1] * len(input_ids) # The mask has 1 for real tokens and 0 for padding tokens. Only real tokens are attended to
        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        init_ids += padding
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(init_ids) == max_seq_length
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        # print('init_ids', init_ids)
        # print('input_ids', input_ids)
        # print('input_mask', input_mask)
        # print('segment_ids', segment_ids)

        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                [str(x) for x in tokens]))
            logger.info("init_ids: %s" % " ".join([str(x) for x in init_ids]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("masked_lm_labels: %s" % " ".join([str(x) for x in masked_lm_labels]))
        features.append(
                InputFeatures(init_ids=init_ids, # original tokens ids
                              input_ids=input_ids, # have mask in the sentences and to ids
                              input_mask=input_mask, # padding 0 and no padding 1
                              segment_ids=segment_ids, # if label=='1' then seg_ids = 1* [length of sentence ]
                              masked_lm_labels=masked_lm_labels, # 做label用， 这样才能去训练
                              token_idx=token_idx)) # 被mask的位置的原来的token的ids
    return features

## Main

In [0]:
def main(train_number):
    parser = argparse.ArgumentParser()

    # Required parameters
    parser.add_argument("--data_dir", default="datasets", type=str,
                        help="The input data dir. Should contain the .tsv files (or other data files) for the task.")
    parser.add_argument("--output_dir", default="aug_data", type=str,
                        help="The output dir for augmented dataset")
    parser.add_argument("--bert_model", default="bert-base-uncased", type=str,
                        help="The path of pretrained bert model.")
    parser.add_argument("--task_name",default="twitter",type=str,
                        help="The name of the task to train.")
    parser.add_argument("--max_seq_length", default=30, type=int,
                        help="The maximum total input sequence length after WordPiece tokenization. \n"
                             "Sequences longer than this will be truncated, and sequences shorter \n"
                             "than this will be padded.")
    parser.add_argument("--do_lower_case", default=True, action='store_true',
                        help="Set this flag if you are using an uncased model.")
    parser.add_argument("--train_batch_size", default=4, type=int,
                        help="Total batch size for training.")
    parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--num_train_epochs", default=10, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--warmup_proportion", default=0.1, type=float,
                        help="Proportion of training to perform linear learning rate warmup for. "
                             "E.g., 0.1 = 10%% of training.")
    parser.add_argument('--seed', type=int, default=42,
                        help="random seed for initialization")

    args, unknown = parser.parse_known_args()
    # print(args.data_dir)
    run_aug(train_number, args, save_every_epoch=True)

In [0]:
class AugProcessor(DataProcessor):
  """Processor for dataset to be augmented."""

  def get_train_examples(self, data_dir):
      """See base class."""
      print('data_dir in augprocessor', data_dir)
      return self._create_examples(
          self._read_csv(os.path.join(data_dir, "train.tsv")), "train")

  def get_dev_examples(self, data_dir):
      """See base class."""
      return self._create_examples(
          self._read_csv(os.path.join(data_dir, "dev.tsv")), "dev")

  def get_labels(self, name):
      """add your dataset here"""
      if name in ['toxic']:
          return ["0", "1"]
      else:
          return ['positive', 'negative', 'neutral']

  def _create_examples(self, lines, set_type):
      """Creates examples for the training and dev sets."""
      examples = []
      for (i, line) in enumerate(lines):
          guid ="%s-%s" % (set_type, i)
          text_a = line[1][0]
          text_b = None
          label = line[1][-1]
          examples.append(
              InputExample(guid, text_a, text_b, label))
      # print('examples:', examples)
      return examples

  @classmethod
  def _read_csv(cls, input_file, quotechar='"'):
      """Reads a comma separated value file."""  
      print('input_file', input_file)
      with open(input_file,"r",encoding='UTF-8') as f:
          # print('input-file', input_file)
          reader = csv.reader(
              f,
              delimiter=",",
              quotechar=quotechar,
              doublequote=True,
              skipinitialspace=False,
              )

          lines = []
          for index, line in enumerate(reader):
              # if index == 500:
              #     break
                # print(line)
              line = ''.join(line).split('\t')
                # print('line', ''.join(line).split('\t'))
              lines.append((index,line))
          del lines[0]
          # print('line', lines)
      print('length of data', len(lines))
      return lines

In [0]:

def remove_wordpiece(str):
    if len(str) > 1:
        for i in range(len(str) - 1, 0, -1):
            if str[i] == '[PAD]':
                str.remove(str[i])
            elif len(str[i]) > 1 and str[i][0] == '#' and str[i][1] == '#':
                str[i - 1] += str[i][2:]
                str.remove(str[i])
    return " ".join(str[1:-1])



def path_setting(task_name = 'twitter', tech_name = 'Bert_ data augmentation-master'):
  abspath = '/content/drive/My Drive/Data/'
  model_path = os.path.join(abspath, tech_name + '/save_model')
  dataset_path = os.path.join(abspath, tech_name + '/datasets/{}/'.format(task_name))
  save_path = os.path.join(abspath, tech_name + '/aug_data/{}/'.format(task_name))
  return model_path, dataset_path, save_path

# train_number = 0
# train_number += 1

## Training

In [0]:
%%time
def run_aug(train_number, args, save_every_epoch=False):
    # Augment the dataset with your own choice of Processer
    processors = {
        "toxic": AugProcessor,
        'twitter': AugProcessor
    }

    task_name = args.task_name
    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    model_path, dataset_path, save_path = path_setting(task_name=task_name)
    args.data_dir = dataset_path
    print('data_dir', type(args.data_dir))
    args.output_dir = save_path
    print('output_dir', args.output_dir)

    processor = processors[task_name]()
    label_list = processor.get_labels(task_name)
    print('label_list', label_list)
    tokenizer = BertTokenizer.from_pretrained(args.bert_model, do_lower_case=args.do_lower_case) 

    train_examples = None
    num_train_steps = None
    print('args.data_dir', args.data_dir)
    train_examples = processor.get_train_examples(args.data_dir)
    
    #dev_examples = processor.get_dev_examples(args.data_dir)
    #train_examples.extend(dev_examples)
    num_train_steps = int(len(train_examples) / args.train_batch_size * args.num_train_epochs) 
    
    # scheduler = WarmupCosineWithHardRestartsSchedule(optimizer=optimizer, warmup_steps=WARMUP_STEPS, t_total=-1)
    #  optimizer = BertAdam(optimizer_grouped_parameters,lr=args.learning_rate,
                        #  warmup=args.warmup_proportion,t_total=t_total)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('device', device)
    #  MODEL_name = "{}/BertForMaskedLM_aug{}_epoch_3".format(task_name.lower(), task_name.lower())
    model = BertForMaskedLM.from_pretrained(args.bert_model)
    
    
    model.bert.embeddings.token_type_embeddings = torch.nn.Embedding(3, 768)
    model.bert.embeddings.token_type_embeddings.weight.data.normal_(mean=0.0, std=0.02)
    
    model.to(device)
    param_optimizer = list(model.named_parameters())
    optimizer = AdamW(model.parameters(), lr=args.learning_rate)
    global_step = 0
    train_features = convert_examples_to_features(
        train_examples, label_list, args.max_seq_length, tokenizer)
    # clear_output()
    print('the training example')
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_examples))
    logger.info("  Batch size = %d", args.train_batch_size)
    logger.info("  Num steps = %d", num_train_steps)

    all_init_ids = torch.tensor([f.init_ids for f in train_features], dtype=torch.long)
    all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)

    all_masked_lm_labels = torch.tensor([f.masked_lm_labels for f in train_features], dtype=torch.long)
    train_data = TensorDataset(all_init_ids, all_input_ids, all_input_mask, all_segment_ids, all_masked_lm_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)
    MASK_id = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
    save_model_dir = model_path
    model.train()

    for e in trange(int(args.num_train_epochs), desc="Epoch"):
      best_accuracy = 0
      avg_loss = 0
        # torch.cuda.empty_cache()
      count = 0
        # shutil.copy(origin_train_path, save_train_path)
        
      for step, batch in enumerate(train_dataloader):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        init_ids, input_ids, input_mask, segment_ids, all_masked_lm_labels = batch
        loss = model(input_ids=input_ids, token_type_ids=segment_ids, attention_mask=input_mask, masked_lm_labels=all_masked_lm_labels)[0]
        # print('loss', loss.item())
        loss.backward()
        avg_loss += loss.item()
        optimizer.step()
        model.zero_grad()

      print('avg_loss: {}'.format(avg_loss / len(train_dataloader)))      
    save_model_name = "Fune-tune_BERT_" + task_name
    save_model_path = os.path.join(save_model_dir, save_model_name)
                # print('where is the model', save_model_path)
    torch.save(model, save_model_path)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.9 µs


In [0]:
if __name__ == "__main__":
  main(train_number)
  train_number +=1

data_dir <class 'str'>
output_dir /content/drive/My Drive/Data/Bert_ data augmentation-master/aug_data/twitter/
label_list ['positive', 'negative', 'neutral']
args.data_dir /content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter/
data_dir in augprocessor /content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter/
input_file /content/drive/My Drive/Data/Bert_ data augmentation-master/datasets/twitter/train.tsv
length of data 41705
device cuda:0
label_map {'positive': 0, 'negative': 1, 'neutral': 2}



Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

the training example



Epoch:  10%|█         | 1/10 [12:51<1:55:38, 771.00s/it]

avg_loss: 4.443307448453273


...

avg_loss: 0.19854988954524802



Epoch:  90%|█████████ | 9/10 [1:56:55<13:01, 781.96s/it]

avg_loss: 0.18293199704937235



Epoch: 100%|██████████| 10/10 [2:09:52<00:00, 779.24s/it]

avg_loss: 0.17232712157343683
